In [ ]:
import math
import re
import json
from typing import List, Dict, Any, Tuple
from llm_utils import LocalLLMClient, setup_logging
from whitebox_calibration import calibrate_expert_prob, find_optimal_temperature

# ============== 配置 ==============
setup_logging(level="INFO")

# 创建全局本地LLM客户端（白盒模型）
client = LocalLLMClient(
    model_id="Qwen/Qwen3-4B-Thinking-2507",  # 本地模型
    local_path="./models/qwen3-4b-thinking",
    device="auto",  # 自动选择GPU/CPU
    max_tokens=200,
    temperature=0.7
)

# === 工具函数 ===
def sigmoid(x):
    return 1 / (1 + math.exp(-x))

def get_yes_no_probabilities_from_distribution(token_distribution: Dict[str, float]) -> Tuple[float, float]:
    """
    从token概率分布中提取Yes/No的概率
    """
    # 初始化概率
    yes_prob = 0.0
    no_prob = 0.0
    
    # 查找各种形式的Yes/No token
    yes_tokens = ["Yes", "yes", "YES", "是", "对的", "正确", "True", "true"]
    no_tokens = ["No", "no", "NO", "否", "不对", "错误", "False", "false"]
    
    # 累加所有Yes相关token的概率
    for token in yes_tokens:
        if token in token_distribution:
            yes_prob += token_distribution[token]
    
    # 累加所有No相关token的概率
    for token in no_tokens:
        if token in token_distribution:
            no_prob += token_distribution[token]
    
    # 如果没有找到任何Yes/No token，检查子串
    if yes_prob == 0 and no_prob == 0:
        for token, prob in token_distribution.items():
            token_lower = token.lower()
            if any(yes_word in token_lower for yes_word in ["yes", "是", "对", "true"]):
                yes_prob += prob
            elif any(no_word in token_lower for no_word in ["no", "否", "错", "false"]):
                no_prob += prob
    
    # 如果还是没有找到，尝试归一化第一个token的概率
    if yes_prob == 0 and no_prob == 0 and token_distribution:
        # 取前两个token的概率
        sorted_tokens = sorted(token_distribution.items(), key=lambda x: x[1], reverse=True)
        if len(sorted_tokens) >= 2:
            # 假设第一个token是"Yes"或"No"
            # 这里我们无法判断，返回0.5/0.5
            return 0.5, 0.5
    
    # 归一化概率
    total = yes_prob + no_prob
    if total > 0:
        yes_prob_normalized = yes_prob / total
        no_prob_normalized = no_prob / total
    else:
        yes_prob_normalized = 0.5
        no_prob_normalized = 0.5
    
    return yes_prob_normalized, no_prob_normalized

def llm_judge_with_token_prob(prompt, x1, x2, n_sample=5, method='token_prob'):
    """
    使用白盒LLM进行判断，基于token概率
    
    参数:
        prompt: 提示词
        x1, x2: 变量名
        n_sample: 采样次数（用于frequency模拟）
        method: 'token_prob' | 'frequency' | 'logit' (保持接口兼容性，实际都使用token概率)
    """
    if method == 'frequency':
        # 频率方法：多次采样，统计Yes/No出现频率
        votes = []
        yes_probs = []
        no_probs = []
        
        for _ in range(n_sample):
            try:
                # 调用本地模型，获取token概率
                response = client.chat(
                    prompt,
                    return_token_probs=True,
                    temperature=0.7  # 添加随机性
                )
                
                # 裁剪思考链
                cropped_response = response.crop_thinking()
                
                # 获取第一个token的概率分布
                token_dist = client.get_token_distributions(
                    cropped_response,
                    top_k=20,
                    skip_thinking=True,
                    skip_zeros=True,
                    zero_threshold=0.001
                )
                
                if len(token_dist) > 0:
                    # 从概率分布中提取Yes/No概率
                    yes_prob, no_prob = get_yes_no_probabilities_from_distribution(token_dist[0])
                    yes_probs.append(yes_prob)
                    no_probs.append(no_prob)
                    
                    # 根据概率投票
                    vote = 1 if yes_prob >= no_prob else 0
                    votes.append(vote)
            except Exception as e:
                print(f"采样失败: {e}")
                continue
        
        if not votes:
            p_yes = 0.5
            p_no = 0.5
        else:
            p_yes = sum(votes) / len(votes)
            p_no = 1 - p_yes
        
        # 计算平均概率
        avg_yes_prob = sum(yes_probs) / len(yes_probs) if yes_probs else p_yes
        avg_no_prob = sum(no_probs) / len(no_probs) if no_probs else p_no
        
        if p_yes >= p_no:
            return {"label": 1, "prob": avg_yes_prob, "yes_prob": avg_yes_prob, "no_prob": avg_no_prob}
        else:
            return {"label": 0, "prob": avg_no_prob, "yes_prob": avg_yes_prob, "no_prob": avg_no_prob}
    
    elif method == 'logit':
        # Logit方法：解析logits并计算概率
        try:
            response = client.chat(prompt, return_token_probs=True)
            text = response.content.strip()
            
            # 尝试从文本中解析logits
            matches = re.findall(r"([-]?[0-9]*\.?[0-9]+)", text)
            if len(matches) >= 2:
                l1, l2 = float(matches[0]), float(matches[1])
                logit_yes, logit_no = l1, l2
            else:
                # 如果无法解析，使用token概率
                return llm_judge_with_token_prob(prompt, x1, x2, n_sample=1, method='token_prob')
            
            p_yes = sigmoid(logit_yes)
            p_no = sigmoid(logit_no)
            
            # 归一化
            total = p_yes + p_no
            if total > 0:
                p_yes_normalized = p_yes / total
                p_no_normalized = p_no / total
            else:
                p_yes_normalized = 0.5
                p_no_normalized = 0.5
            
            if p_yes_normalized >= p_no_normalized:
                return {"label": 1, "prob": p_yes_normalized, "yes_prob": p_yes_normalized, "no_prob": p_no_normalized}
            else:
                return {"label": 0, "prob": p_no_normalized, "yes_prob": p_yes_normalized, "no_prob": p_no_normalized}
                
        except Exception as e:
            print(f"Logit方法失败: {e}, 回退到token_prob方法")
            return llm_judge_with_token_prob(prompt, x1, x2, n_sample=1, method='token_prob')
    
    else:  # token_prob 方法
        # 使用白盒模型的token概率
        all_yes_probs = []
        all_no_probs = []
        
        for i in range(max(1, n_sample)):  # 至少采样1次
            try:
                # 调用本地模型，获取token概率
                response = client.chat(
                    prompt,
                    return_token_probs=True,
                    temperature=0.1 if n_sample == 1 else 0.7  # 单次采样降低随机性
                )
                
                # 裁剪思考链
                cropped_response = response.crop_thinking()
                
                # 获取第一个token的概率分布
                token_dist = client.get_token_distributions(
                    cropped_response,
                    top_k=20,
                    skip_thinking=True,
                    skip_zeros=True,
                    zero_threshold=0.001
                )
                
                if len(token_dist) > 0:
                    # 从概率分布中提取Yes/No概率
                    yes_prob, no_prob = get_yes_no_probabilities_from_distribution(token_dist[0])
                    
                    all_yes_probs.append(yes_prob)
                    all_no_probs.append(no_prob)
                    
                    if n_sample > 1:
                        print(f"  采样{i+1}: P(Yes)={yes_prob:.4f}, P(No)={no_prob:.4f}")
                        
            except Exception as e:
                print(f"采样{i+1}失败: {e}")
                continue
        
        # 如果没有成功采样，使用默认值
        if not all_yes_probs:
            avg_yes_prob = 0.5
            avg_no_prob = 0.5
        else:
            # 计算平均概率
            avg_yes_prob = sum(all_yes_probs) / len(all_yes_probs)
            avg_no_prob = sum(all_no_probs) / len(all_no_probs)
        
        if n_sample > 1:
            print(f"  平均概率: P(Yes)={avg_yes_prob:.4f}, P(No)={avg_no_prob:.4f}")
        
        # 根据平均概率决定标签
        if avg_yes_prob >= avg_no_prob:
            return {"label": 1, "prob": avg_yes_prob, "yes_prob": avg_yes_prob, "no_prob": avg_no_prob}
        else:
            return {"label": 0, "prob": avg_no_prob, "yes_prob": avg_yes_prob, "no_prob": avg_no_prob}

# === MoE 专家定义 ===
CAUSAL_EXPERTS = {
    "graph_theory": {
        "name": "因果图论专家",
        "description": "专门分析因果图结构，精通d-分离、路径阻断、后门准则等图论概念。推理方式：系统检查所有可能的路径，分析路径上的变量类型和连接方式，使用严谨的图论推理链条。",
        "specialty": "路径分析、环路检测、d-分离判断",
        "reasoning_style": "结构化图遍历",
        "output_format": "基于图结构的二值判断"
    },
    "statistical": {
        "name": "计量统计专家", 
        "description": "专注于统计检验和概率独立性分析，擅长相关性分析、条件独立性检验、混淆变量检测。推理方式：考虑样本分布、统计显著性、置信区间等统计概念。",
        "specialty": "独立性检验、相关性分析、混淆检测",
        "reasoning_style": "概率统计推理",
        "output_format": "基于统计证据的概率判断"
    },
    "domain_knowledge": {
        "name": "领域先验专家",
        "description": "基于现实世界知识和科学常识进行因果推理，考虑时间顺序、物理可能性、生物学机制等约束。推理方式：结合文献证据、科学理论和常识性约束。",
        "specialty": "机制分析、时序推理、现实约束",
        "reasoning_style": "基于证据的归纳推理", 
        "output_format": "基于领域知识的合理性判断"
    },
    "counterfactual": {
        "name": "反事实干预专家",
        "description": "从干预和潜在结果角度分析因果关系，考虑do-calculus、随机化实验理想情况。推理方式：构建反事实场景，分析干预后的可能变化。",
        "specialty": "干预分析、潜在结果、do算子",
        "reasoning_style": "反事实思维实验",
        "output_format": "基于干预推理的因果判断"
    },
    "temporal_dynamics": {
        "name": "时间动态专家",
        "description": "专门分析时间顺序和动态过程，强调原因必须发生在结果之前，考虑延迟效应和动态反馈。推理方式：严格检查时间顺序，分析因果链的时间特性。",
        "specialty": "时序分析、动态过程、延迟效应",
        "reasoning_style": "时间序列推理", 
        "output_format": "基于时间顺序的因果判断"
    },
    "mechanism_modeling": {
        "name": "机制建模专家", 
        "description": "专注于因果机制的可解释性建模，分析中间变量、中介效应和机制路径。推理方式：构建机制框图，分析变量间的功能关系。",
        "specialty": "中介分析、机制路径、功能关系",
        "reasoning_style": "机制分解建模",
        "output_format": "基于机制完整性的判断"
    },
    "robustness_analysis": {
        "name": "稳健性检验专家",
        "description": "专门评估因果关系的稳健性和敏感性，考虑不同假设下的结果稳定性。推理方式：进行敏感性分析，检验边界条件和假设变化的影响。",
        "specialty": "敏感性分析、稳健检验、边界情况",
        "reasoning_style": "多情景验证",
        "output_format": "基于稳健性评估的判断"
    }
}

# === Router 函数（增强版）===
def expert_router(question_type: str, x1: str, x2: str) -> List[str]:
    """
    根据问题类型和变量特征选择最相关的专家
    """
    # 基础路由规则
    routing_rules = {
        "backdoor_path": [
            "graph_theory", "statistical", "counterfactual", "temporal_dynamics", 
            "mechanism_modeling", "robustness_analysis", "domain_knowledge"
        ],
        "independence": [
            "statistical", "graph_theory", "counterfactual", "robustness_analysis",
            "temporal_dynamics", "mechanism_modeling", "domain_knowledge"
        ],
        "latent_confounder": [
            "domain_knowledge", "statistical", "mechanism_modeling", "counterfactual",
            "robustness_analysis", "graph_theory", "temporal_dynamics"
        ],
        "causal_direction": [
            "temporal_dynamics", "domain_knowledge", "counterfactual", "mechanism_modeling",
            "statistical", "graph_theory", "robustness_analysis"
        ]
    }
    
    # 获取基础专家列表
    base_experts = routing_rules.get(question_type, list(CAUSAL_EXPERTS.keys()))
    
    # 使用门诊LLM agent来智能选择专家
    try:
        clinic_recommendation = clinic_agent_recommend(question_type, x1, x2, base_experts)
        return clinic_recommendation
    except Exception as e:
        print(f"门诊agent路由失败: {e}，使用基础路由")
        # 失败时返回基础专家列表的前3个
        return base_experts[:3]

def clinic_agent_recommend(question_type: str, x1: str, x2: str, base_experts: List[str]) -> List[str]:
    """
    门诊LLM agent：根据具体变量和问题类型推荐最相关的专家
    """
    # 使用独立的LLM客户端进行推荐
    clinic_client = LocalLLMClient(
        model_id="Qwen/Qwen3-4B-Thinking-2507",
        local_path="./models/qwen3-4b-thinking",
        device="auto",
        max_tokens=200,
        temperature=0.1
    )
    
    # 构建专家选择提示
    experts_description = "\n".join([
        f"- {expert}: {CAUSAL_EXPERTS[expert]['description']}" 
        for expert in base_experts
    ])
    
    clinic_prompt = f"""
作为因果推断门诊专家，你需要为以下因果分析任务选择最合适的专家组合：

**分析任务**: {question_type}
**变量对**: {x1} 和 {x2}

**可用专家列表**:
{experts_description}

**选择要求**:
1. 根据变量内容和问题类型，选择最相关的3个专家
2. 按相关性从高到低排序
3. 确保专家视角的多样性（不要选择推理方式相似的专家）
4. 考虑变量的领域特性（医学、经济、社会等）

请按照以下格式输出：
最终推荐专家: 专家1, 专家2, 专家3

**注意**: 只输出专家名称，用逗号分隔，不要添加其他文字。
"""
    
    # 调用LLM获取推荐
    response = clinic_client.chat(clinic_prompt)
    response_text = response.content.strip()
    print(f"门诊agent原始响应: {response_text}")
    
    # 解析返回的专家列表
    recommended_experts = parse_clinic_recommendation(response_text, base_experts)
    
    print(f"门诊agent推荐: {recommended_experts}")
    
    return recommended_experts

def parse_clinic_recommendation(response_text: str, base_experts: List[str]) -> List[str]:
    """
    解析门诊agent的推荐结果
    """
    # 方法1: 查找"最终推荐专家"后的内容
    if "最终推荐专家" in response_text:
        parts = response_text.split("最终推荐专家")
        if len(parts) > 1:
            expert_line = parts[1].strip().lstrip(":").strip()
            return extract_experts_from_line(expert_line, base_experts)
    
    # 方法2: 查找最后一行
    lines = [line.strip() for line in response_text.split('\n') if line.strip()]
    if lines:
        last_line = lines[-1]
        experts = extract_experts_from_line(last_line, base_experts)
        if len(experts) >= 2:
            return experts
    
    # 方法3: 在整个文本中搜索专家名称
    found_experts = []
    for expert in base_experts:
        if expert in response_text:
            found_experts.append(expert)
    
    if len(found_experts) >= 2:
        return found_experts[:3]  # 取前3个找到的专家
    
    # 如果所有方法都失败，返回基础专家前3个
    print(f"门诊agent解析不充分，使用基础专家: {base_experts[:3]}")
    return base_experts[:3]

def extract_experts_from_line(line: str, base_experts: List[str]) -> List[str]:
    """从一行文本中提取专家名称"""
    experts = []
    
    # 清理行内容
    clean_line = line.replace('：', ':').replace('，', ',').replace(' ', '')
    
    # 多种分割方式尝试
    separators = [',', '、', ';', '，']
    
    for sep in separators:
        if sep in clean_line:
            parts = [part.strip() for part in clean_line.split(sep)]
            break
    else:
        parts = [clean_line]
    
    for part in parts:
        clean_part = part.lower().replace('专家', '').replace('expert', '').strip()
        
        # 直接匹配专家名称
        for expert in base_experts:
            if (expert in clean_part or 
                expert.replace('_', ' ') in clean_part or
                CAUSAL_EXPERTS[expert]['name'] in part):
                if expert not in experts:
                    experts.append(expert)
                    break
        
        if len(experts) >= 3:
            break
    
    return experts

# === 专家提示创建函数 ===
def create_expert_prompt(base_prompt: str, expert_type: str, x1: str, x2: str) -> str:
    """
    为不同专家创建专业化的prompt
    """
    expert_info = CAUSAL_EXPERTS[expert_type]
    
    expert_specific_prompts = {
        "graph_theory": f"""作为{expert_info['name']}，请严格遵循以下专业分析框架：

{expert_info['description']}

专业特长：{expert_info['specialty']}
推理风格：{expert_info['reasoning_style']}
输出要求：基于图结构的二值判断

分析步骤：
1. 构建因果图模型，识别所有可能的路径
2. 应用d-分离准则分析路径阻塞情况  
3. 检查后门路径、前门路径和混杂路径
4. 基于图结构做出明确的二值判断

请严格按照图论原理进行分析，直接输出是或否（Yes/No）。\n\n{base_prompt}""",

        "statistical": f"""作为{expert_info['name']}，请严格遵循以下专业分析框架：

{expert_info['description']}

专业特长：{expert_info['specialty']}
推理风格：{expert_info['reasoning_style']}
输出要求：基于统计证据的二值判断

分析步骤：
1. 评估变量间的统计相关性
2. 考虑条件独立性和混淆因素
3. 分析统计显著性和置信度
4. 基于概率证据做出明确的二值判断

请基于统计原理进行严谨分析，直接输出是或否（Yes/No）。\n\n{base_prompt}""",

        "domain_knowledge": f"""作为{expert_info['name']}，请严格遵循以下专业分析框架：

{expert_info['description']}

专业特长：{expert_info['specialty']}
推理风格：{expert_info['reasoning_style']}
输出要求：基于领域知识的二值判断

分析步骤：
1. 调用相关领域的科学知识和常识
2. 考虑物理/生物/社会机制的合理性
3. 评估时间顺序和现实约束条件
4. 基于先验知识做出明确的二值判断

请结合现实世界知识进行推理，直接输出是或否（Yes/No）。\n\n{base_prompt}""",

        "counterfactual": f"""作为{expert_info['name']}，请严格遵循以下专业分析框架：

{expert_info['description']}

专业特长：{expert_info['specialty']}
推理风格：{expert_info['reasoning_style']}
输出要求：基于干预推理的二值判断

分析步骤：
1. 构建干预场景（do-操作）
2. 比较实际结果与反事实结果
3. 分析潜在结果分布
4. 基于干预效应做出明确的二值判断

请使用反事实推理进行分析，直接输出是或否（Yes/No）。\n\n{base_prompt}""",

        "temporal_dynamics": f"""作为{expert_info['name']}，请严格遵循以下专业分析框架：

{expert_info['description']}

专业特长：{expert_info['specialty']}
推理风格：{expert_info['reasoning_style']}
输出要求：基于时间顺序的二值判断

分析步骤：
1. 严格检查原因和结果的时间顺序
2. 分析延迟效应和动态过程
3. 考虑时间序列的因果结构
4. 基于时间约束做出明确的二值判断

请重点分析时间维度，直接输出是或否（Yes/No）。\n\n{base_prompt}""",

        "mechanism_modeling": f"""作为{expert_info['name']}，请严格遵循以下专业分析框架：

{expert_info['description']}

专业特长：{expert_info['specialty']}
推理风格：{expert_info['reasoning_style']}
输出要求：基于机制完整性的二值判断

分析步骤：
1. 识别可能的中间机制和中介变量
2. 分析因果链的功能完整性
3. 评估机制路径的合理性
4. 基于机制可解释性做出明确的二值判断

请专注于机制分析，直接输出是或否（Yes/No）。\n\n{base_prompt}""",

        "robustness_analysis": f"""作为{expert_info['name']}，请严格遵循以下专业分析框架：

{expert_info['description']}

专业特长：{expert_info['specialty']}
推理风格：{expert_info['reasoning_style']}
输出要求：基于稳健性评估的二值判断

分析步骤：
1. 测试不同假设条件下的结果稳定性
2. 进行敏感性分析和边界检验
3. 评估结论的稳健程度
4. 基于稳健性评估做出明确的二值判断

请重点分析结论的可靠性，直接输出是或否（Yes/No）。\n\n{base_prompt}"""
    }
    
    return expert_specific_prompts.get(expert_type, base_prompt)

# === MoE 集成函数 ===
def aggregate_expert_judgments(expert_results: List[Dict]) -> Dict:
    """
    整合多个专家的判断结果
    """
    if not expert_results:
        return {"label": 0, "prob": 0.5}
    
    # 简单加权平均
    total_prob_yes = 0
    total_weight = 0
    
    for result in expert_results:
        weight = result.get("confidence", 1.0)
        # 使用yes_prob字段（从token概率得来）
        prob_yes = result.get("calibrated_yes_prob", result.get("yes_prob", result["prob"] if result["label"] == 1 else 1 - result["prob"]))
        total_prob_yes += prob_yes * weight
        total_weight += weight
    
    aggregated_prob_yes = total_prob_yes / total_weight if total_weight > 0 else 0.5
    
    if aggregated_prob_yes >= 0.5:
        return {"label": 1, "prob": aggregated_prob_yes, "yes_prob": aggregated_prob_yes, "no_prob": 1 - aggregated_prob_yes}
    else:
        return {"label": 0, "prob": 1 - aggregated_prob_yes, "yes_prob": aggregated_prob_yes, "no_prob": 1 - aggregated_prob_yes}

# === MoE判断函数 ===
def run_step_with_moe(base_prompt: str, x1: str, x2: str, question_type: str, method: str = 'token_prob', true_labels: list = None) -> Dict:
    """
    使用MoE架构运行单个判断步骤（新增校准参数）
    """
    # 1. 路由选择专家
    selected_experts = expert_router(question_type, x1, x2)
    print(f"为问题 '{question_type}' 选择的专家: {selected_experts}")
    
    # 2. 执行专家判断
    expert_results = []
    expert_yes_probs = []  # 新增：收集原始Yes概率，用于找最优温度
    for expert in selected_experts:
        expert_prompt = create_expert_prompt(base_prompt, expert, x1, x2)
        try:
            # 使用本地白盒模型的token概率方法
            result = llm_judge_with_token_prob(expert_prompt, x1, x2, n_sample=2, method=method)
            result["expert"] = expert
            result["confidence"] = 1.0
            expert_results.append(result)
            expert_yes_probs.append(result["yes_prob"])  # 新增：收集原始概率
            print(f"专家 {expert} 判断完成: label={result['label']}, P(Yes)={result.get('yes_prob', result['prob']):.4f}, P(No)={result.get('no_prob', 1-result['prob']):.4f}")
        except Exception as e:
            print(f"专家 {expert} 执行失败: {e}")
            continue
    
    # 3. 如果没有专家成功，使用默认方法
    if not expert_results:
        print("所有专家执行失败，使用默认方法")
        return llm_judge_with_token_prob(base_prompt, x1, x2, n_sample=3, method=method)
    
    # ========== 新增：白箱校准逻辑 ==========
    # 找最优温度（无真实标签时用1.0）
    opt_temp = find_optimal_temperature(expert_yes_probs, true_labels)
    print(f"最优校准温度: {opt_temp}")
    
    # 对每个专家的概率进行温度缩放校准
    for result in expert_results:
        yes_prob = result["yes_prob"]
        no_prob = result["no_prob"]
        # 校准概率
        calib_yes, calib_no, entropy = calibrate_expert_prob(yes_prob, no_prob, opt_temp)
        # 保存校准结果
        result["calibrated_yes_prob"] = calib_yes
        result["calibrated_no_prob"] = calib_no
        result["entropy"] = entropy  # 熵值（衡量不确定性）
        result["calibration_temp"] = opt_temp
        # 打印校准日志
        print(f"专家 {result['expert']} 校准后: P(Yes)={calib_yes:.4f}, P(No)={calib_no:.4f}, 熵值={entropy:.4f}")
    # ======================================
    
    # 4. 整合专家意见（自动使用校准后的概率）
    final_result = aggregate_expert_judgments(expert_results)
    final_result["expert_results"] = expert_results
    # 新增：保存校准相关的汇总信息
    final_result["calibration_temp"] = opt_temp
    final_result["avg_entropy"] = np.mean([r["entropy"] for r in expert_results]) if expert_results else 0.5
    
    print(f"专家整合结果: label={final_result['label']}, P(Yes)={final_result['yes_prob']:.4f}, P(No)={final_result['no_prob']:.4f}")
    print(f"校准汇总: 最优温度={opt_temp:.2f}, 平均熵值={final_result['avg_entropy']:.4f}")
    return final_result

# === 具体判断函数（新增true_labels参数）===
def check_backdoor(x1, x2, all_variables, method='token_prob', true_labels=None):
    base_prompt = f"""在因果推断中，考虑以下所有变量：{all_variables}

请判断在这些变量中，变量 {x1} 和 {x2} 之间是否存在 back-door path（后门路径）。

后门路径是指从 {x1} 到 {x2} 的路径，其中包含指向 {x1} 的箭头，且这条路径没有被阻断。

让我们一步步思考，然后直接输出是或否（Yes/No）。"""
    return run_step_with_moe(base_prompt, x1, x2, "backdoor_path", method, true_labels=true_labels)

def check_independence_after_block(x1, x2, all_variables, method='token_prob', true_labels=None):
    base_prompt = f"""在因果推断中，考虑以下所有变量：{all_variables}

如果阻断了 {x1} 和 {x2} 之间的所有 back-door path，那么 {x1} 与 {x2} 是否条件独立？

让我们一步步思考，然后直接输出是或否（Yes/No）。"""
    return run_step_with_moe(base_prompt, x1, x2, "independence", method, true_labels=true_labels)

def check_latent_confounder_after_block(x1, x2, all_variables, method='token_prob', true_labels=None):
    base_prompt = f"""在因果推断中，考虑以下所有变量：{all_variables}

阻断了 {x1} 和 {x2} 之间的所有 back-door path 后，是否仍然存在未观察到的潜在混杂因子同时影响 {x1} 和 {x2}？

让我们一步步思考，然后直接输出是或否（Yes/No）。"""
    return run_step_with_moe(base_prompt, x1, x2, "latent_confounder", method, true_labels=true_labels)

def check_causal_direction_after_block(x1, x2, all_variables, method='token_prob', true_labels=None):
    base_prompt = f"""在因果推断中，考虑以下所有变量：{all_variables}

阻断了 {x1} 和 {x2} 之间的所有 back-door path 后，请判断 {x1} 是否因果导致 {x2}？

让我们一步步思考，然后直接输出是或否（Yes/No）。"""
    return run_step_with_moe(base_prompt, x1, x2, "causal_direction", method, true_labels=true_labels)

def check_independence(x1, x2, all_variables, method='token_prob', true_labels=None):
    base_prompt = f"""在因果推断中，考虑以下所有变量：{all_variables}

请判断变量 {x1} 和 {x2} 是否独立？

让我们一步步思考，然后直接输出是或否（Yes/No）。"""
    return run_step_with_moe(base_prompt, x1, x2, "independence", method, true_labels=true_labels)

def check_latent_confounder(x1, x2, all_variables, method='token_prob', true_labels=None):
    base_prompt = f"""在因果推断中，考虑以下所有变量：{all_variables}

请判断变量 {x1} 和 {x2} 之间是否存在未观察到的潜在混杂因子？

让我们一步步思考，然后直接输出是或否（Yes/No）。"""
    return run_step_with_moe(base_prompt, x1, x2, "latent_confounder", method, true_labels=true_labels)

def check_causal_direction(x1, x2, all_variables, method='token_prob', true_labels=None):
    base_prompt = f"""在因果推断中，考虑以下所有变量：{all_variables}

请判断 {x1} 是否因果导致 {x2}？

让我们一步步思考，然后直接输出是或否（Yes/No）。"""
    return run_step_with_moe(base_prompt, x1, x2, "causal_direction", method, true_labels=true_labels)

def load_true_labels_from_json(json_path: str, x1: str, x2: str) -> dict:
    """
    从JSON数据集（element1/element2/causal）读取真实标签，映射到树状推理步骤
    
    参数：
        json_path: JSON文件路径（如"./causal_500.json"）
        x1: 当前分析的变量1
        x2: 当前分析的变量2
    返回：
        true_labels_dict: 树状推理各步骤的真实标签字典（仅映射因果方向步骤）
    """
    # 1. 读取JSON文件
    with open(json_path, "r", encoding="utf-8") as f:
        data = json.load(f)  # 假设JSON是列表格式：[{"element1":"xxx","element2":"xxx","causal":0}, ...]
    
    # 2. 匹配变量对（兼容element1/element2顺序颠倒）
    matched_item = None
    for item in data:
        e1, e2 = item["element1"], item["element2"]
        # 匹配当前x1/x2（顺序无关）
        if (e1 == x1 and e2 == x2) or (e1 == x2 and e2 == x1):
            matched_item = item
            break
    
    # 3. 映射标签：JSON的causal → 树状推理的"因果方向判断"步骤
    true_labels_dict = {}
    if matched_item:
        causal_label = matched_item["causal"]  # 0/1
        # 映射到树状推理的两个因果方向步骤（有后门路径/无后门路径）
        true_labels_dict["x->y_after_block"] = [causal_label]    # 有后门路径时的因果方向
        true_labels_dict["x->y_no_backdoor"] = [causal_label]    # 无后门路径时的因果方向
        print(f"✅ 从JSON匹配到{x1}-{x2}的真实因果标签：{causal_label}（1=x→y，0=y→x/无因果）")
    else:
        print(f"⚠️ 在JSON中未找到{x1}-{x2}的匹配数据，校准使用默认温度")
    
    return true_labels_dict

# === 完整的树查询函数 ===
def tree_query(x1, x2, all_variables, method='token_prob', true_labels_dict: dict = None):
    """
    基于树状逻辑的因果方向查询器（支持校准）
    - true_labels_dict: 各步骤的真实标签，格式{"backdoor_path": [1/0], "independent_after_block": [1/0], ...}
    """
    log = []

    # Step 1: 是否存在 backdoor path?
    print("=== Step 1: 检查后门路径 ===")
    # 传递对应步骤的真实标签（用于校准）
    true_labels = true_labels_dict.get("backdoor_path") if true_labels_dict else None
    res_backdoor = check_backdoor(x1, x2, all_variables, method, true_labels=true_labels)
    log.append(("backdoor_path", res_backdoor))

    if res_backdoor["label"] == 1:
        print("存在后门路径，进入阻断后分析路径")
        # Step 2: 阻断路径后是否独立？
        print("=== Step 2: 阻断后检查独立性 ===")
        true_labels = true_labels_dict.get("independent_after_block") if true_labels_dict else None
        res_ind = check_independence_after_block(x1, x2, all_variables, method, true_labels=true_labels)
        log.append(("independent_after_block", res_ind))
        if res_ind["label"] == 1:
            return {"relation": "independent", "confidence": res_ind["yes_prob"], "log": log}

        # Step 3: 是否存在潜在混杂因子？
        print("=== Step 3: 检查潜在混杂因子 ===")
        true_labels = true_labels_dict.get("latent_confounder_after_block") if true_labels_dict else None
        res_latent = check_latent_confounder_after_block(x1, x2, all_variables, method, true_labels=true_labels)
        log.append(("latent_confounder_after_block", res_latent))
        if res_latent["label"] == 1:
            return {"relation": "x<->y", "confidence": res_latent["yes_prob"], "log": log}

        # Step 4: 判断方向 (x→y?)
        print("=== Step 4: 判断因果方向 ===")
        true_labels = true_labels_dict.get("x->y_after_block") if true_labels_dict else None
        res_dir = check_causal_direction_after_block(x1, x2, all_variables, method, true_labels=true_labels)
        log.append(("x->y_after_block", res_dir))
        if res_dir["label"] == 1:
            return {"relation": "x->y", "confidence": res_dir["yes_prob"], "log": log}
        else:
            return {"relation": "y->x", "confidence": res_dir["no_prob"], "log": log}

    else:
        print("不存在后门路径，进入直接分析路径")
        # 不存在 backdoor path
        true_labels = true_labels_dict.get("independent_no_backdoor") if true_labels_dict else None
        res_ind = check_independence(x1, x2, all_variables, method, true_labels=true_labels)
        log.append(("independent_no_backdoor", res_ind))
        if res_ind["label"] == 1:
            return {"relation": "independent", "confidence": res_ind["yes_prob"], "log": log}

        true_labels = true_labels_dict.get("latent_confounder_no_backdoor") if true_labels_dict else None
        res_latent = check_latent_confounder(x1, x2, all_variables, method, true_labels=true_labels)
        log.append(("latent_confounder_no_backdoor", res_latent))
        if res_latent["label"] == 1:
            return {"relation": "x<->y", "confidence": res_latent["yes_prob"], "log": log}

        true_labels = true_labels_dict.get("x->y_no_backdoor") if true_labels_dict else None
        res_dir = check_causal_direction(x1, x2, all_variables, method, true_labels=true_labels)
        log.append(("x->y_no_backdoor", res_dir))
        if res_dir["label"] == 1:
            return {"relation": "x->y", "confidence": res_dir["yes_prob"], "log": log}
        else:
            return {"relation": "y->x", "confidence": res_dir["no_prob"], "log": log}

if __name__ == "__main__":
    # ===================== 基础配置 =====================
    # 定义分析的变量对和所有变量
    x1 = "冰淇淋销量"
    x2 = "溺水人数"
    all_variables = ["冰淇淋销量", "溺水人数", "温度"]
    
    # JSON数据集路径（确保路径正确）
    json_path = "./causal_500.json"
    
    # ===================== 读取真实标签（优先用JSON） =====================
    # 从JSON读取对应变量对的真实标签（用于校准）
    true_labels_dict = load_true_labels_from_json(
        json_path=json_path,
        x1=x1,
        x2=x2
    )
    
    # 【可选】如果没有JSON数据，取消下面注释，使用手动定义的标签（二选一）
    # true_labels_dict = {
    #     "backdoor_path": [1],
    #     "independent_after_block": [1],
    #     "latent_confounder_after_block": [0],
    #     "x->y_after_block": [0],
    #     "independent_no_backdoor": [1],
    #     "latent_confounder_no_backdoor": [0],
    #     "x->y_no_backdoor": [0]
    # }
    
    # ===================== 选择单个方法运行（核心修改） =====================
    # 仅使用一个方法（可选：token_prob/frequency/logit，推荐token_prob）
    method = "token_prob"
    
    # ===================== 执行因果推断分析 =====================
    print("开始因果推断分析（本地白盒模型 + token概率 + 温度缩放校准）")
    print("=" * 60)
    print(f"\n使用方法: {method}")
    print("-" * 40)
    
    # 执行树状推理（传入真实标签校准）
    result = tree_query(
        x1=x1,
        x2=x2,
        all_variables=all_variables,
        method=method,
        true_labels_dict=true_labels_dict
    )
    
    # ===================== 打印结果（简化，仅展示单个方法） =====================
    print(f"\n=== {method}方法最终结果 ===")
    print(f"变量对: {x1} ↔ {x2}")
    print(f"因果关系: {result['relation']}")
    print(f"判断置信度: {result['confidence']:.4f}")
    
    # 详细执行日志（保留完整信息）
    print(f"\n=== 详细执行日志 ===")
    for step_name, step_result in result["log"]:
        print(f"\n🔍 步骤: {step_name}")
        print(f"  最终判断: {'是（Yes）' if step_result['label'] == 1 else '否（No）'}")
        print(f"  原始P(Yes): {step_result.get('yes_prob', step_result['prob']):.4f}")
        print(f"  原始P(No): {step_result.get('no_prob', 1-step_result['prob']):.4f}")
        print(f"  校准温度: {step_result.get('calibration_temp', 1.0):.2f}")
        print(f"  平均熵值: {step_result.get('avg_entropy', 0.0):.4f}")
        
        # 专家详情（保留）
        if "expert_results" in step_result:
            print(f"  🧑‍🏫 参与专家详情:")
            for expert_result in step_result["expert_results"]:
                expert_name = expert_result.get("expert", "unknown")
                yes_prob = expert_result.get("yes_prob", expert_result["prob"] if expert_result["label"] == 1 else 1 - expert_result["prob"])
                no_prob = expert_result.get("no_prob", 1 - yes_prob)
                calib_yes = expert_result.get("calibrated_yes_prob", yes_prob)
                calib_no = expert_result.get("calibrated_no_prob", no_prob)
                entropy = expert_result.get("entropy", 0.0)
                print(f"    - {expert_name}: 原始(Yes={yes_prob:.4f}, No={no_prob:.4f}) → 校准后(Yes={calib_yes:.4f}, No={calib_no:.4f})，熵值={entropy:.4f}")
    
    # ===================== 收尾 =====================
    print(f"\n{'='*60}")
    print("✅ 分析完成！")
    # 清理模型释放内存
    client.unload_model()
    print("🗑️ 模型已卸载，资源释放完成")

d:\25877\WORKS\LLM-for-Causality\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


12-12 14:22:33 [INFO] llm_utils.local_client - Model not found locally or invalid, downloading: Qwen/Qwen3-4B-Thinking-2507
12-12 14:22:33 [INFO] llm_utils.local_client - Downloading model from https://huggingface.co: Qwen/Qwen3-4B-Thinking-2507
12-12 14:22:33 [INFO] llm_utils.local_client - Target directory: ./models/qwen3-4b-thinking


d:\25877\WORKS\LLM-for-Causality\.venv\Lib\site-packages\huggingface_hub\file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
d:\25877\WORKS\LLM-for-Causality\.venv\Lib\site-packages\huggingface_hub\file_download.py:979: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(
Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]'(ProtocolError('Connection aborted.', ConnectionAbortedError(10053, '你的主机中的软件中止了一个已建立的连接。', None, 10053, None)), '(Request ID: e61b8bf5-9319-45ea-bdaf-f870271c

In [ ]:
from itertools import combinations

def compute_all_causal_relations(variables, method='probability'):
    """
    计算图中每两个变量之间的因果关系，使用tree_query函数。
    
    输出:
        {
            (x1, x2): {
                'relation': 'x->y' | 'y->x' | 'x<->y' | 'independent',
                'confidence': float,
                'log': [(step_name, {'label': int, 'prob': float}), ...]
            },
            ...
        }
    """
    all_relations = {}

    # 生成所有变量的组合 C(n, 2)
    for x1, x2 in combinations(variables, 2):
        # 进行 tree_query
        result = tree_query(x1, x2, method)
        
        # 存储结果
        all_relations[(x1, x2)] = result

    return all_relations


In [ ]:
variables = ['气温', '冰淇淋销量', '溺水人数']
relations = compute_all_causal_relations(variables)

for (x1, x2), relation in relations.items():
    print(f"Relation between {x1} and {x2}: {relation['relation']} (Confidence: {relation['confidence']})")


NameError: name 'compute_all_causal_relations' is not defined